# Prepare CNN Data

## Setup

In [1]:
import ee
#ee.Authenticate()
ee.Initialize()

In [2]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time
from datetime import datetime
import glob

DROPBOX_DIR = cf.DROPBOX_DIRECTORY

In [3]:
def decode_fn_uid(record_bytes):
    return tf.io.parse_single_example(
        # Data
        record_bytes,

        # Schema
        {"uid": tf.io.FixedLenFeature([], dtype=tf.string)}
    )

def extract_uid(TF_FILES):
    actual_values = []
    for batch in tf.data.TFRecordDataset([TF_FILES]).map(decode_fn_uid):
        value = batch['uid'].numpy()
        actual_values.append(value)

    return actual_values

## Parameters

In [4]:
# Datasets -------------------------------------
SURVEY_NAME = 'LSMS'

VERSION = 1
if VERSION == 1:

    SATELLITE         = 's2' 
    OUTCOME_VAR       = "viirs" 
    UNDERSAMPLE_INDIA = True
    
if VERSION == 2:

    SATELLITE         = 'landsat' 
    OUTCOME_VAR       = "ntlharmon" 
    UNDERSAMPLE_INDIA = True

# Processing data ------------------------------
SKIP_IF_SCRAPED = True ## Skip if filename has already been created
CHECK_IF_UID_SCRAPED = False ## Load data already scraped and skip if scraped; add date to filename
IGNORE_ERRORS = False ## Load dataset of errors and remove from ones to scrape

CHUNK_SIZE = 1 # Number of observtaions to scrape in GEE at any given time

# Parameters based on dataset ---------------------
if SATELLITE == 's2':
    KERNEL_SIZE = 224
elif SATELLITE == 'landsat':
    KERNEL_SIZE = 224 #167
elif SATELLITE == 'landsat_7':
    KERNEL_SIZE = 224 #167

print(KERNEL_SIZE)

224


In [5]:
# Directory to store tfrecords
out_path = os.path.join(DROPBOX_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE + '_' + OUTCOME_VAR + '_underia' + str(UNDERSAMPLE_INDIA),
            'tfrecords')

out_path_errors = os.path.join(DROPBOX_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE + '_' + OUTCOME_VAR + '_underia' + str(UNDERSAMPLE_INDIA))

## Load Data

In [6]:
### Load data
if UNDERSAMPLE_INDIA == True:
    UNDERSAMPLE_INDIA_str = "TRUE"
else:
    UNDERSAMPLE_INDIA_str = "FALSE"
    
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 
                                     'data_for_cnn_' + OUTCOME_VAR + '_iaunder' + UNDERSAMPLE_INDIA_str + '_' + SATELLITE + '.csv'))

### If sentinel, only use most recent
if SATELLITE == 's2':
    survey_df = survey_df[survey_df.most_recent_survey == True]
        
### N Observations      
print(survey_df.shape)
print(survey_df.ntl_group.value_counts())

(3939, 9)
0    1716
2     669
3     630
1     597
4     327
Name: ntl_group, dtype: int64


### Check if UID Already Scraped

(1) Filter surveys to those that have been scraped, and (2) add date/time to filename (so process of checking if file has been scraped doesnt skip it)

In [7]:
if IGNORE_ERRORS:
    error_files = glob.glob(out_path_errors + '/*.csv')
    error_df = pd.concat([pd.read_csv(f) for f in error_files])

    survey_df = survey_df[~survey_df['uid'].isin(error_df['uid'])]

In [8]:
## Check if everything processed, ignoring errors
if False:
    tf_paths = glob.glob(out_path + '/*.tfrecord')
    processed_uids = extract_uid(tf_paths)

    ## List of IDs already processed
    processed_uids = [x.decode('utf-8') for x in processed_uids]

    ## Subset survey to uids not scraped
    survey_df = survey_df[~survey_df['uid'].isin(processed_uids)]

In [9]:
if CHECK_IF_UID_SCRAPED:

    tf_paths = glob.glob(out_path + '/*.tfrecord')
    processed_uids = extract_uid(tf_paths)
    
    ## List of IDs already processed
    processed_uids = [x.decode('utf-8') for x in processed_uids]
    
    ## Subset survey to uids not scraped
    survey_df = survey_df[~survey_df['uid'].isin(processed_uids)]
    
    ## Change name of tfrecords
    txt_to_add = datetime.now().strftime("%Y%m%d%H%M%S")
    
    survey_df['tfrecord_name'] = survey_df.apply(lambda row: row['tfrecord_name'].replace('.tfrecord', "_" + txt_to_add + '.tfrecord'), axis='columns')

### Skip tfrecords already processed

In [10]:
survey_df.head()

,uid,GID_2,year,most_recent_survey,ntl_group,longitude,latitude,tfrecord_name,use_for_cnn
0,id_1114,BFA.8.1_1,2018,True,0,0.378829,13.020676,forcnn_test_BF_1_1_all.tfrecord,yes
1,id_801,BFA.5.3_1,2018,True,0,-2.259757,11.270638,forcnn_test_BF_2_1_all.tfrecord,yes
2,id_2352,ETH.8.3_1,2018,True,0,39.021503,3.609384,forcnn_test_ET_5_1_all.tfrecord,yes
3,id_1416,CIV.2.2_1,2018,True,0,-6.678438,5.443886,forcnn_test_CI_2_1_all.tfrecord,yes
4,id_3038,MWI.22.3_1,2019,True,0,34.647461,-14.480580,forcnn_test_MW_1_1_all.tfrecord,yes


In [11]:
survey_df.shape

(3939, 9)

In [12]:
# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

len(tf_record_list)

tf_record_list

['forcnn_test_BF_1_1_all.tfrecord',
 'forcnn_test_BF_2_1_all.tfrecord',
 'forcnn_test_BF_3_1_all.tfrecord',
 'forcnn_test_BF_4_1_all.tfrecord',
 'forcnn_test_BF_5_1_all.tfrecord',
 'forcnn_test_BJ_1_1_all.tfrecord',
 'forcnn_test_BJ_2_1_all.tfrecord',
 'forcnn_test_BJ_3_1_all.tfrecord',
 'forcnn_test_BJ_4_1_all.tfrecord',
 'forcnn_test_BJ_5_1_all.tfrecord',
 'forcnn_test_CI_1_1_all.tfrecord',
 'forcnn_test_CI_2_1_all.tfrecord',
 'forcnn_test_CI_3_1_all.tfrecord',
 'forcnn_test_CI_4_1_all.tfrecord',
 'forcnn_test_CI_5_1_all.tfrecord',
 'forcnn_test_ET_1_1_all.tfrecord',
 'forcnn_test_ET_2_1_all.tfrecord',
 'forcnn_test_ET_3_1_all.tfrecord',
 'forcnn_test_ET_4_1_all.tfrecord',
 'forcnn_test_ET_5_1_all.tfrecord',
 'forcnn_test_MW_1_1_all.tfrecord',
 'forcnn_test_MW_2_1_all.tfrecord',
 'forcnn_test_MW_3_1_all.tfrecord',
 'forcnn_test_MW_4_1_all.tfrecord',
 'forcnn_test_MW_5_1_all.tfrecord',
 'forcnn_test_TG_1_1_all.tfrecord',
 'forcnn_test_TG_2_1_all.tfrecord',
 'forcnn_test_TG_3_1_all.tfr

In [13]:
if os.path.exists(out_path) == False:
    os.makedirs(out_path)

In [14]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]
    
print(len(tf_record_list))

42


In [15]:
# https://gist.github.com/erdemarslan/3ec02009f38f8df84c8e4807e7954af3
if False:
    import urllib3

    def check_internet_conn():
        http = urllib3.PoolManager(timeout=3.0)
        r = http.request('GET', 'google.com', preload_content=False)
        code = r.status
        r.release_conn()
        if code == 200:
            return True
        else:
            return False

## Query Data

In [16]:
## Blank error dataframe
errors_df = pd.DataFrame()

## Error file name
now = datetime.now()
current_time = now.strftime("%d_%m_%y_%H_%M_%S")
error_file_name = 'errors_' + current_time + '.csv'

if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.

        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            

            try:

                time.sleep(3)
                print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                proto_examples_all.extend(proto_examples_i)

            except:

                print("Error ---")
                print(survey_df_yeari_chunki['uid'])

                errors_df = errors_df.append(survey_df_yeari_chunki[['uid']], ignore_index = True)
                errors_df.to_csv(os.path.join(out_path_errors, error_file_name))

                time.sleep(15)
                pass

        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")



/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_30230/407197897.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Putting 47 observations into forcnn_train_ET_4_1_all.tfrecord
Observation: 0/47


2023-07-02 00:09:43.658337: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Observation: 1/47
Observation: 2/47
Observation: 3/47
Observation: 4/47
Observation: 5/47
Observation: 6/47
Observation: 7/47
Observation: 8/47
Observation: 9/47
Observation: 10/47
Observation: 11/47
Observation: 12/47
Observation: 13/47
Observation: 14/47
Observation: 15/47
Observation: 16/47
Observation: 17/47
Observation: 18/47
Observation: 19/47
Observation: 20/47
Observation: 21/47
Observation: 22/47
Observation: 23/47
Observation: 24/47
Observation: 25/47
Observation: 26/47
Observation: 27/47
Observation: 28/47
Observation: 29/47
Observation: 30/47
Observation: 31/47
Observation: 32/47
Observation: 33/47
Observation: 34/47
Observation: 35/47
Observation: 36/47
Observation: 37/47
Observation: 38/47
Observation: 39/47
Observation: 40/47
Observation: 41/47
Observation: 42/47
Observation: 43/47
Observation: 44/47
Observation: 45/47
Observation: 46/47
/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/LSMS/FinalData/Individual Datasets/cnn_s2_viirs

/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_30230/407197897.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


...

Observation: 0/49
Observation: 1/49
Observation: 2/49
Observation: 3/49
Observation: 4/49
Observation: 5/49
Observation: 6/49
Observation: 7/49
Observation: 8/49
Observation: 9/49
Observation: 10/49
Observation: 11/49
Observation: 12/49
Observation: 13/49
Observation: 14/49
Observation: 15/49
Observation: 16/49
Observation: 17/49
Observation: 18/49
Observation: 19/49
Observation: 20/49
Observation: 21/49
Observation: 22/49
Observation: 23/49
Observation: 24/49
Observation: 25/49
Observation: 26/49
Observation: 27/49
Observation: 28/49
Observation: 29/49
Observation: 30/49
Observation: 31/49
Observation: 32/49
Observation: 33/49
Observation: 34/49
Observation: 35/49
Observation: 36/49
Observation: 37/49
Observation: 38/49
Observation: 39/49
Observation: 40/49
Observation: 41/49
Observation: 42/49
Observation: 43/49
Observation: 44/49
Observation: 45/49
Observation: 46/49
Observation: 47/49
Observation: 48/49
/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satelli

/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_30230/407197897.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Observation: 0/39
Observation: 1/39
Observation: 2/39
Observation: 3/39
Observation: 4/39
Observation: 5/39
Observation: 6/39
Observation: 7/39
Observation: 8/39
Observation: 9/39
Observation: 10/39
Observation: 11/39
Observation: 12/39
Observation: 13/39
Observation: 14/39
Observation: 15/39
Observation: 16/39
Observation: 17/39
Observation: 18/39
Observation: 19/39
Observation: 20/39
Observation: 21/39
Observation: 22/39
Observation: 23/39
Observation: 24/39
Observation: 25/39
Observation: 26/39
Observation: 27/39
Observation: 28/39
Observation: 29/39
Observation: 30/39
Observation: 31/39
Observation: 32/39
Observation: 33/39
Observation: 34/39
Observation: 35/39
Observation: 36/39
Observation: 37/39
Observation: 38/39
/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/LSMS/FinalData/Individual Datasets/cnn_s2_viirs_underiaTrue/tfrecords/nocnn_TG_4_1_all.tfrecord
Success \o/
Putting 34 observations into nocnn_TG_5_1_all.tfrecord


/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/ipykernel_30230/407197897.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)


Observation: 0/34
Observation: 1/34
Observation: 2/34
Observation: 3/34
Observation: 4/34
Observation: 5/34
Observation: 6/34
Observation: 7/34
Observation: 8/34
Observation: 9/34
Observation: 10/34
Observation: 11/34
Observation: 12/34
Observation: 13/34
Observation: 14/34
Observation: 15/34
Observation: 16/34
Observation: 17/34
Observation: 18/34
Observation: 19/34
Observation: 20/34
Observation: 21/34
Observation: 22/34
Observation: 23/34
Observation: 24/34
Observation: 25/34
Observation: 26/34
Observation: 27/34
Observation: 28/34
Observation: 29/34
Observation: 30/34
Observation: 31/34
Observation: 32/34
Observation: 33/34
/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/LSMS/FinalData/Individual Datasets/cnn_s2_viirs_underiaTrue/tfrecords/nocnn_TG_5_1_all.tfrecord
Success \o/
